# Hikurangi Ruptures Demo using ipyvolume

Use the **pyvolume** library to produce an interactive 3D visualisation of Hikurangi w Ruptures

In [12]:
import ipyvolume as ipv
# import ipyvolume
import requests
import numpy as np

In [13]:
!pip install geopandas >/dev/null
!pip install nest_asyncio > /dev/null
import geopandas as gpd

In [14]:
#for data animations
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [15]:
# tiles  - choose your resolution
url30 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_outlines_30.zip?raw=true"
url10 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_outlines.zip?raw=true"
tile_outlines_nztm = gpd.read_file(url10).to_crs(epsg=2193)

## Build the tri-polygon coords and indices need for ipyvolume

In [16]:
def quadPolyToTriPoly(coords):
    assert len(coords) == 5
    x = [g[0] for g in coords[:-1]]
    y = [g[1] for g in coords[:-1]]
    z = [g[2] for g in coords[:-1]]
    return x, y, z

# build the fault section tile coords lists 
x,y,z = [], [], []
for bdry in tile_outlines_nztm.geometry.boundary:
    x0,y0,z0 = quadPolyToTriPoly(list(bdry.coords))
    x.extend(x0)
    y.extend(y0)
    z.extend(z0)

In [17]:
# add the triangle indices to the dataframe
def calculateFaceIdices(geometry):
    #using this trick https://stackoverflow.com/a/18317089 to get the index
    idx = int(geometry.name) *4
    faces = [[idx, idx+1, idx+2], [idx, idx+2, idx+3]]
    return faces
    
tile_outlines_nztm['triangles'] = tile_outlines_nztm['geometry'].to_frame().apply(calculateFaceIdices, axis=1)

## ruptures

In [18]:
from io import BytesIO
import codecs
import requests
module_uri = "https://github.com/GNS-Science/nshm-nz-opensha/raw/python_ruptures_101/python/src/fault_section.py"
url = requests.get(module_uri, allow_redirects=True)
bytesio_object = BytesIO(url.content)
# Write the stuff
with open("fault_section.py", "wb") as f:
    f.write(bytesio_object.getbuffer())

import fault_section
from fault_section import FaultSubSectionFactory, SheetFault

In [19]:
# url = "https://github.com/GNS-Science/nshm-nz-opensha/blob/python_ruptures_101/data/subduction/tile_parameters_30.csv?raw=true"
url30 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_parameters_30.csv?raw=true"
url10 = "https://github.com/GNS-Science/eq-fault-geom/blob/chris_ruptures_101/data/subduction/tile_parameters.csv?raw=true"
tile_params = BytesIO(requests.get(url10).content)
StreamReader = codecs.getreader('utf-8')
wrapper_file = StreamReader(tile_params)

factory = FaultSubSectionFactory()
sf = SheetFault("Hikurangi")\
		.build_surface_from_csv(factory, wrapper_file)

In [20]:
# Variations on 'nearly rectangular' rupture sets ..
#
# scale: determines ratio of rectangle to original tile size (10km*10km).
# aspect: ratio of 'along-strike' tiles to 'down-dip' tiles per rectangle.
# interhttps://hub.gke.mybinder.org/user/chrisbc-379a8fb-0a027b8c1b4eff1-75eu0sje/notebooks/Hikurangi%20Rupture%20Demo.ipynb#val: how many tiles to advance both col-wise and row-wise
# min_fill_factor: how many tiles are needed to make up a valid 'rectangle'
#
spec0 = dict(scale=3, aspect=2)
spec1 = dict(scale=3, aspect=4, interval=2, min_fill_factor=0.5) #default min_fill_factor is 0.75
spec2 = dict(scale=8, aspect=2, interval=4, min_fill_factor=0.55)
spec3 = dict(scale=16, aspect=1, interval=4, min_fill_factor=0.33)
spec4 = dict(scale=16, aspect=2, interval=4, min_fill_factor=0.33)
spec5 = dict(scale=3, aspect=12, interval=2, min_fill_factor=0.55)

#select the 'spec' to animate here...
selected_spec = spec1

#now build the rupture set 
ruptures = [rupt for rupt in sf.get_rupture_ids(selected_spec)]

## visualisation

In [21]:
#draw the tetrahedron
fig = ipv.figure()
all_tris = np.array(tile_outlines_nztm['triangles'].to_list(), dtype='uint32')
mesh = ipv.plot_trisurf(x=x, y=y, z=z, triangles=all_tris, color='lightblue')
faces = ipv.plot_trisurf(x, y, z, triangles=[[]], color='red')
ipv.ylim( min(y), max(y))
ipv.xlim( min(x), max(x))
ipv.zlim(-250e3, 0)
ipv.show()

In [ ]:
# use async co-routines to animate the rupture data 
async def new_faces(n):
    #apply the new filter
    rupt_tris = tile_outlines_nztm[tile_outlines_nztm["FID"].isin(ruptures[n])]
    #convert series to np.array and update the faces
    faces.triangles = np.array(rupt_tris['triangles'].to_list(), dtype='uint32')   

for n in range(len(ruptures)):
    task = asyncio.create_task(new_faces(n))
    await task
    await asyncio.sleep(0.2)
